In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import erf
import mibitrans as mbt


The governing equation for advective-dispersive transport with adsorption and decay given by Bear (1979) is:

\begin{equation}\tag{1}
    R\frac{\partial C}{\partial t} = -v\frac{\partial C}{\partial x} + D_{x}\frac{\partial ^2 C}{\partial x^2} + D_y \frac{\partial^2 C}{\partial y^2} + D_z \frac{\partial^2 C}{\partial z^2} - \mu C
\end{equation}

With $0\leq x \leq \infty$, $-\infty \leq y \leq \infty$ and $-\infty \leq z \leq \infty$. With the contaminant source modeled as a continuous planar block located at x=0 and of width Y and thickness Z, one gets the following exact analytical equation given by Wexler (1992):

\begin{align}\tag{2}
    C(x,y,z,t) = & \frac{C_{0}x}{8\sqrt{\pi \alpha_x v/R}}
    \int_0^t \tau^{-3/2} \exp \left(-\mu\tau - \frac{(x-v\tau/R)^2}{4\alpha_xv\tau/R}\right) \cdot\\
    & \quad \left[\operatorname{erf} \left( \frac{y+Y/2}{2\sqrt{D_y\tau/R}} \right) -
    \operatorname{erf} \left( \frac{y-Y/2}{2\sqrt{D_y\tau/R}} \right)\right]\cdot \nonumber \\
    & \quad\left[\operatorname{erf} \left( \frac{z+Z/2}{2\sqrt{D_z\tau/R}} \right) -
    \operatorname{erf} \left( \frac{z-Z/2}{2\sqrt{D_z\tau/R}} \right)\right] d\tau
    \nonumber
\end{align}

Where $\tau$ is the integration variable for time. There is however, no full exact analytical solution of this equation.

The paper of Domenico and Robbins (1985) suggests an approximation to the 3D solution by using the product of the 1D solution in each direction:

\begin{equation}\tag{3}
    C(x,y,z,t) \approx C(x,t) \cdot C(y,t) \cdot C(z,t)
\end{equation}

Individually, each 1D solution can be derived in a full analytical equation, leading to:

\begin{align}\tag{4}
    C(x, y, z, t) &= \frac{C_{0}}{8} \left( \exp \left[ \frac{xv\left(1-\sqrt{1+4\mu R D_x/v^2}\right)}{2D_x}\right] \right. \\
    &\quad \quad \cdot \operatorname{erfc} \left[ \frac{x - \frac{vt}{R}\sqrt{1+4\mu R D_x/v^2}}{2\sqrt{D_x t / R }} \right] \\
    &\quad \ \, + \exp \left[ \frac{xv\left(1+\sqrt{1+4\mu R D_x/v^2}\right)}{2D_x}\right] \\
    &\quad \quad \cdot \left. \operatorname{erfc} \left[ \frac{x + \frac{vt}{R}\sqrt{1+4\mu R D_x/v^2}}{2\sqrt{D_x t/R }} \right] \right) \\
    &\quad \cdot \left( \operatorname{erf} \left[ \frac{y + Y/2}{2\sqrt{D_y t / R}} \right] - \operatorname{erf} \left[ \frac{y - Y/2}{2\sqrt{D_y t / R)}} \right] \right) \\
    &\quad \cdot \left( \operatorname{erf} \left[ \frac{z + Z/2}{2\sqrt{D_z t / R)}} \right] - \operatorname{erf} \left[ \frac{z - Z/2}{2\sqrt{D_z t / R}} \right] \right)
\end{align}

However, the equation presented in Domenico (1987) is different:

\begin{align}\tag{5}
    C(x, y, z, t) &= \frac{C_{0}}{8} \left( \exp \left[ \frac{xv\left(1-\sqrt{1+4\mu R D_x/v^2}\right)}{2D_x}\right] \right. \\
    &\quad \quad \cdot \operatorname{erfc} \left[ \frac{x - \frac{vt}{R}\sqrt{1+4\mu R D_x/v^2}}{2\sqrt{D_x t / R }} \right] \\
    &\quad \cdot \left( \operatorname{erf} \left[ \frac{y + Y/2}{2\sqrt{\alpha_y x}} \right] - \operatorname{erf} \left[ \frac{y - Y/2}{2\sqrt{\alpha_y x)}} \right] \right) \\
    &\quad \cdot \left( \operatorname{erf} \left[ \frac{z + Z/2}{2\sqrt{\alpha_z x)}} \right] - \operatorname{erf} \left[ \frac{z - Z/2}{2\sqrt{\alpha_z x}} \right] \right)
\end{align}

Notice the missing second exponent and erfc term of the $C(x,t)$ solution, and that in the transverse dispersion terms, $D_{y,z}t/R$ is replaced by $\alpha_{y,z}x$. The first difference is due to a truncation of the x-term in the solution. But equation \tag{5} can be untruncated and include the additional x-term without any issue, and is implemented as such in the Anatrans solution of this package. The second difference is accomplished by substituting $t$ for $x/v$ (technically $xR/v$ since we consider retardation). Assuming the effect of molecular diffusion in transport is negligible, $D_{y,z} = \alpha_{y,z}v$. Therefore making $2\sqrt{D_{y,z} t / R)} = 2\sqrt{\alpha_{y,z}v\frac{xR}{v}/R} = 2\sqrt{\alpha_{y,z}x}$. $x/v$ can be seen as the time it takes for the advective front to reach position $x$ under flow velocity $v$.

Albeit this substitution in equation 4 lacks a mathematical basis, as put in West et al. (2007), it is essential for approximating the exact equation. For a continuous source, $\lim_{t \to \infty}C(x=0,y,z,t)$ should be $C_0$, absent any source decay or depletion. However, for equation 4, $\lim_{t \to \infty}C(x=0,y,z,t) = 0$, as both transverse dispersion terms go to 0 for $t \to \infty$. Therefore $C(x,y,z,t) \not \approx C(x,t) \cdot C(y,t) \cdot C(z,t)$, meaning equation 4 (presented as equation 22 in West et al. (2007) as the 'final approximate nontruncated solution') is not a correct approximate solution in the first place. This is shown below by comparing equation 4 and 5 to the exact solution, equation 2. To more accurately compare the two equations, equation 5 is used in its untruncated form.

In [ ]:

hydro = mbt.HydrologicalParameters(
    velocity=0.3, #[m/d]
    porosity=0.25, #[-]
    alpha_x=4, #[m]
    alpha_y=0.4, #[m]
    alpha_z=0.04 #[m]
)

att = mbt.AttenuationParameters(
    bulk_density=1.7, #[g/m^3]
    partition_coefficient=38, #[m^3/g]
    fraction_organic_carbon=5.7e-5, #[-]
    diffusion=0, #[m2/day]
    half_life=0 #[days]
)

source = mbt.SourceParameters(
    source_zone_boundary=np.array([11]),
    source_zone_concentration=np.array([14]),
    depth=3,
    total_mass="inf"
)

model = mbt.ModelParameters(
    model_length = 450, #[m]
    model_width = 100, #[m]
    model_time = 4 * 365, #[days]
    dx = 1, #[m]
    dy = 1, #[m]
    dt = 25 #[days]
)

In [ ]:
class AnatransWithoutSubstitution(mbt.Anatrans):
    """Anatrans model class without substitution."""

    def _equation_term_z(self, ttt):
        inner_term = self._src_pars.depth / (2 * np.sqrt(self._hyd_pars.alpha_z * self.rv * ttt))
        return erf(inner_term) - erf(-inner_term)

    def _equation_term_y(self, i, yyy, ttt):
        div_term = 2 * np.sqrt(self._hyd_pars.alpha_y * self.rv * ttt)
        term = erf((yyy + self.y_source[i]) / div_term) - erf((yyy - self.y_source[i]) / div_term)
        term[np.isnan(term)] = 0
        return term

    def _calculate_concentration_for_all_xyt(self, xxx, yyy, ttt):
        cxyt = 0
        decay_sqrt = np.sqrt(1 + 4 * (self._decay_rate - self.k_source) * self._hyd_pars.alpha_x / self.rv)
        x_term = self._equation_term_x(xxx, ttt, decay_sqrt)
        additional_x = self._equation_term_additional_x(xxx, ttt, decay_sqrt)
        z_term = self._equation_term_z(ttt)
        source_decay = self._equation_term_source_depletion(xxx, ttt)
        for i in range(len(self.c_source)):
            y_term = self._equation_term_y(i, yyy,ttt)
            cxyt_step = 1 / 8 * self.c_source[i] * source_decay * (x_term + additional_x) * y_term * z_term
            cxyt += cxyt_step
        if self._mode == "instant_reaction":
            self.cxyt_noBC = cxyt.copy()
            cxyt -= self.biodegradation_capacity
            cxyt = np.where(cxyt < 0, 0, cxyt)
        self.has_run = True
        return cxyt

In [ ]:
ana_object = mbt.Anatrans(hydro, att, source, model)
ana_results = ana_object.run()
alt_disp_object = AnatransWithoutSubstitution(hydro, att, source, model)
alt_disp_results = alt_disp_object.run()
mbt_object = mbt.Mibitrans(hydro, att, source, model)
mbt_results = mbt_object.run()

In [ ]:
mbt_results.centerline(time=182, color="green", label="eq2, t=182days")
mbt_results.centerline(time=365, color="blue", label="eq2, t=365days")
mbt_results.centerline(time=3*365, color="red", label="eq2, t=1095days")
ana_results.centerline(time=182, color="limegreen", linestyle="-.", label="eq5, t=182days")
ana_results.centerline(time=365, color="cornflowerblue", linestyle="-.", label="eq5, t=365days")
ana_results.centerline(time=3*365, color="orangered", linestyle="-.", label="eq5, t=1095days")
alt_disp_results.centerline(time=182, color="lightgreen", linestyle=":", label="eq4, t=182days")
alt_disp_results.centerline(time=365, color="lightblue", linestyle=":", label="eq4, t=365days")
alt_disp_results.centerline(time=3*365, color="tomato", linestyle=":", label="eq4, t=1095days")

plt.title("Centerline plot of various models")
plt.legend()
plt.show()

In [ ]:
mbt_results.transverse(time=182, x_position=75, color="green", label="eq2, t=182days")
mbt_results.transverse(time=365, x_position=75, color="blue", label="eq2, t=365days")
mbt_results.transverse(time=3*365, x_position=75, color="red", label="eq2, t=1095days")
ana_results.transverse(time=182, x_position=75, color="limegreen", linestyle="-.", label="ana, t=182days")
ana_results.transverse(time=365, x_position=75, color="cornflowerblue", linestyle="-.", label="ana, t=365days")
ana_results.transverse(time=3*365, x_position=75, color="orangered", linestyle="-.", label="ana, t=1095days")
alt_disp_results.transverse(time=182, x_position=75, color="lightgreen", linestyle=":", label="eq4, t=182days")
alt_disp_results.transverse(time=365, x_position=75, color="lightblue", linestyle=":", label="eq4, t=365days")
alt_disp_results.transverse(time=3*365, x_position=75, color="tomato", linestyle=":", label="eq4, t=1095days")
plt.ylim(0, 10)
plt.title("Transverse plot of various models, at x=75m")
plt.legend()
plt.show()

As these graphs show, equation 4, without substitution, does not come close to the exact solution of equation 2 for larger times. Equation 5, with substitution has a small error with the exact solution, but follows the same general trend. As equation 4 does not correspond with the boundary condition $\lim_{t \to \infty}C(x=0,y,z,t) = C_0$, source concentrations do not stay constant. The expressions for transverse dispersion in equation 4 do not involve any representation of x-position and therefore can not abide to a boundary condition at x=0.

Instead of equation 3, the solution from Domenico (1987), is more akin to $C(x,y,z,t) \approx C(x,t) \cdot C(x,y) \cdot (x,z)$. But these solutions can not be mathematically derived from the governing equations. Therefore derivation of the Domenico solution is best represented by starting at equation 2. Assume that transverse dispersion is not dependent on time, but on position along the flow direction. Diffusion is removed from the dispersion coefficient, since it is independent from flow velocity and position. The time integration variable $\tau$ therefore is converted into $\frac{xR}{v}$, resolving to:
\begin{align}\tag{6}
    C(x,y,z,t) = & \frac{C_{0}x}{8\sqrt{\pi \alpha_x v/R}}
    \int_0^t \tau^{-3/2} \exp \left(-\mu\tau - \frac{(x-v\tau/R)^2}{4\alpha_xv\tau/R}\right) \cdot\\
    & \quad \left[\operatorname{erf} \left( \frac{y+Y/2}{2\sqrt{\alpha_y x}} \right) -
    \operatorname{erf} \left( \frac{y-Y/2}{2\sqrt{\alpha_y x}} \right)\right]\cdot \nonumber \\
    & \quad\left[\operatorname{erf} \left( \frac{z+Z/2}{2\sqrt{\alpha_z x}} \right) -
    \operatorname{erf} \left( \frac{z-Z/2}{2\sqrt{\alpha_z x}} \right)\right] d\tau
    \nonumber
\end{align}

Now independent of time, the transverse dispersion terms can be taken outside of the integral. Resulting in:

\begin{align}\tag{4}
    C(x, y, z, t) &= \frac{C_{0}}{8} \left( \exp \left[ \frac{xv\left(1-\sqrt{1+4\mu R D_x/v^2}\right)}{2D_x}\right] \right. \\
    &\quad \quad \cdot \operatorname{erfc} \left[ \frac{x - \frac{vt}{R}\sqrt{1+4\mu R D_x/v^2}}{2\sqrt{D_x t / R }} \right] \\
    &\quad \ \, + \exp \left[ \frac{xv\left(1+\sqrt{1+4\mu R D_x/v^2}\right)}{2D_x}\right] \\
    &\quad \quad \cdot \left. \operatorname{erfc} \left[ \frac{x + \frac{vt}{R}\sqrt{1+4\mu R D_x/v^2}}{2\sqrt{D_x t/R }} \right] \right) \\
    &\quad \cdot \left( \operatorname{erf} \left[ \frac{y + Y/2}{2\sqrt{\alpha_y x}} \right] - \operatorname{erf} \left[ \frac{y - Y/2}{2\sqrt{\alpha_y x}} \right] \right) \\
    &\quad \cdot \left( \operatorname{erf} \left[ \frac{z + Z/2}{2\sqrt{\alpha_z x}} \right] - \operatorname{erf} \left[ \frac{z - Z/2}{2\sqrt{\alpha_z x}} \right] \right)
\end{align}

It should be noted that equation 5 also does not respect the boundary condition $\lim_{t \to \infty}C(x=0,y,z,t) = C_0$ for smaller times due to the truncation, see the example below:

In [ ]:
bio_object = mbt.Bioscreen(hydro, att, source, model)
bio_results = bio_object.run()

In [ ]:
plot_times = [25,50,100,200]
mbt_color = ["blue", "deepskyblue", "lime", "yellowgreen"]
bio_color = ["lightblue", "skyblue", "lightgreen", "lawngreen"]
for i in range(len(plot_times)):
    mbt_results.centerline(time=plot_times[i], color = mbt_color[i],
                           label=f"eq 2, t={plot_times[i]} days")
    bio_results.centerline(time=plot_times[i], linestyle = "-.", color = bio_color[i],
                           label=f"eq 5, t={plot_times[i]} days")
plt.legend()
plt.xlim(-10,125)
plt.title("Difference in source concentration for different transport equations")
plt.show()